In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')
np.random.seed(0)
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [71]:
missing_value = df.isnull().sum()
missing_value[70:82]

ScreenPorch         0
PoolArea            0
PoolQC           1453
Fence            1179
MiscFeature      1406
MiscVal             0
MoSold              0
YrSold              0
SaleType            0
SaleCondition       0
SalePrice           0
dtype: int64

In [4]:
total = np.prod(df.shape)
missing = missing_value.sum()
percent_missing = missing/total*100
print(percent_missing)

6.620158971757145


In [5]:
columns_with_na_dropped = df.dropna(axis=1)
columns_with_na_dropped.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [85]:
lot_frontage = df['LotFrontage']
lot_frontage.fillna(lot_frontage.median(), inplace=True)


In [78]:
df[['GarageQual', 'Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish']]=df[['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual']].fillna('None')
df[['GarageCond', 'PoolQC', 'Fence', 'MiscFeature']]  = df[['GarageCond', 'PoolQC', 'Fence', 'MiscFeature']].fillna('None')

In [ ]:
df['MasVnrArea']=df['MasVnrArea'].fillna(0.0)

In [55]:
df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])

In [63]:
df.loc[df['GarageType'].isna(), 'GarageYrBlt']=0

In [88]:
df.dtypes[0:10]

Id               int64
MSSubClass       int64
MSZoning        object
LotFrontage    float64
LotArea          int64
Street          object
Alley           object
LotShape        object
LandContour     object
Utilities       object
dtype: object

In [94]:
def find_outliers(df):
    outlier_indices = {}
    for col in df.select_dtypes(['float64', 'int64']).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        higher_bound = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower_bound) | (df[col] > higher_bound)].index.tolist()
        if outliers:
            outlier_indices[col] = outliers
    return outlier_indices


In [98]:
out = find_outliers(df)
for column, indices in out.items():
    print(f"Column {column} has {len(indices)} outliers")

Column MSSubClass has 103 outliers
Column LotFrontage has 106 outliers
Column LotArea has 69 outliers
Column OverallQual has 2 outliers
Column OverallCond has 125 outliers
Column YearBuilt has 7 outliers
Column BsmtFinSF1 has 7 outliers
Column BsmtFinSF2 has 167 outliers
Column BsmtUnfSF has 29 outliers
Column TotalBsmtSF has 61 outliers
Column 1stFlrSF has 20 outliers
Column 2ndFlrSF has 2 outliers
Column LowQualFinSF has 26 outliers
Column GrLivArea has 31 outliers
Column BsmtFullBath has 1 outliers
Column BsmtHalfBath has 82 outliers
Column BedroomAbvGr has 35 outliers
Column KitchenAbvGr has 68 outliers
Column TotRmsAbvGrd has 30 outliers
Column Fireplaces has 5 outliers
Column GarageYrBlt has 81 outliers
Column GarageCars has 5 outliers
Column GarageArea has 21 outliers
Column WoodDeckSF has 32 outliers
Column OpenPorchSF has 77 outliers
Column EnclosedPorch has 208 outliers
Column 3SsnPorch has 24 outliers
Column ScreenPorch has 116 outliers
Column PoolArea has 7 outliers
Column 